# Import Dependencies

In [5]:
import requests
import time
import json
from tqdm import tqdm 
import pandas as pd
from pathlib import Path

In [ ]:

TMDB_API_KEY = ''



BASE_URL = "https://api.themoviedb.org/3"

In [ ]:
import requests
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
import time

def fetch_tmdb_details(tmdb_id, api_key=TMDB_API_KEY, retries=3):
    if pd.isna(tmdb_id) or tmdb_id == 0:
        return None
    
    url = f"{BASE_URL}/movie/{int(tmdb_id)}"
    params = {
        'api_key': api_key,
        'language': 'en-US',
        'append_to_response': 'credits,keywords'
    }
    
    # Setup session with retries and longer timeout
    session = requests.Session()
    retry_strategy = Retry(
        total=retries,
        backoff_factor=1,          # wait 1s, 2s, 4s between retries
        status_forcelist=[429, 500, 502, 503, 504],
        allowed_methods=["GET"]
    )
    adapter = HTTPAdapter(max_retries=retry_strategy)
    session.mount("https://", adapter)
    
    try:
        response = session.get(url, params=params, timeout=30)  # ← increased to 30s
        response.raise_for_status()
        data = response.json()
        
        result = {
            'overview': data.get('overview', ''),
            'keywords': ' '.join([k['name'] for k in data.get('keywords', {}).get('keywords', [])]),
            'poster_path': data.get('poster_path'),
            'directors': ', '.join([c['name'] for c in data.get('credits', {}).get('crew', []) if c['job'] == 'Director']),
            'top_cast': ', '.join([c['name'] for c in data.get('credits', {}).get('cast', [])[:5]]),
            'release_date': data.get('release_date', ''),
            'vote_average': data.get('vote_average')
        }
        return result
    
    except requests.exceptions.RequestException as e:
        print(f"Error fetching {tmdb_id}: {e}")
        return None

In [11]:
# Load your enriched movies from earlier
movies_content = pd.read_csv('../data/ml-latest-small/processed/movies_content.csv')  # or use the df you have

# Filter to movies with tmdbId
movies_to_enrich = movies_content[movies_content['tmdbId'].notna()].copy()
movies_to_enrich['tmdbId'] = movies_to_enrich['tmdbId'].astype(int)

print(f"Movies with TMDB ID: {len(movies_to_enrich)} / {len(movies_content)}")

#For testing: start small (e.g. first 200-500)
movies_to_enrich = movies_to_enrich.head(300)
enriched_data = []

for idx, row in tqdm(movies_to_enrich.iterrows(), total=len(movies_to_enrich)):
    details = fetch_tmdb_details(row['tmdbId'])
    if details:
        details['movieId'] = row['movieId']  # key to merge back
        enriched_data.append(details)
    time.sleep(0.25)  # ~4 req/sec → safe under 40/10s limit

# Convert to DF and merge back
tmdb_df = pd.DataFrame(enriched_data)
movies_fully_enriched = movies_content.merge(tmdb_df, on='movieId', how='left')

# Fill NaNs
movies_fully_enriched['overview'] = movies_fully_enriched['overview'].fillna('')
movies_fully_enriched['keywords'] = movies_fully_enriched['keywords'].fillna('')

# Update content_text with new rich features
movies_fully_enriched['content_text'] = (
    movies_fully_enriched['genres'].str.replace('|', ' ', regex=False) + ' ' +
    movies_fully_enriched['title'].str.replace(r'\(\d{4}\)', '', regex=True).str.strip() + ' ' +
    movies_fully_enriched['all_tags'] + ' ' +
    movies_fully_enriched['overview'] + ' ' +
    movies_fully_enriched['keywords'] + ' ' +
    movies_fully_enriched['directors'] + ' ' +
    movies_fully_enriched['top_cast']
).str.lower().str.strip().str.replace(r'\s+', ' ', regex=True)  # clean extra spaces

print("Updated content_text sample:\n", movies_fully_enriched['content_text'].head(3))



Movies with TMDB ID: 9734 / 9742


  0%|                                                                                          | 0/300 [00:00<?, ?it/s]

Error fetching 862: HTTPSConnectionPool(host='api.themoviedb.org', port=443): Read timed out. (read timeout=10)


 80%|████████████████████████████████████████████████████████████████▎               | 241/300 [08:57<02:18,  2.35s/it]

Error fetching 38722: HTTPSConnectionPool(host='api.themoviedb.org', port=443): Read timed out. (read timeout=10)


 88%|██████████████████████████████████████████████████████████████████████▍         | 264/300 [10:09<01:21,  2.27s/it]

Error fetching 161495: HTTPSConnectionPool(host='api.themoviedb.org', port=443): Read timed out.


100%|████████████████████████████████████████████████████████████████████████████████| 300/300 [11:44<00:00,  2.35s/it]

Updated content_text sample:
 0                                                  NaN
1    adventure children fantasy jumanji fantasy mag...
2    comedy romance grumpier old men moldy old a fa...
Name: content_text, dtype: str


In [12]:
movies_fully_enriched.to_csv('../data/ml-latest-small/processed/movies_fully_enriched_300.csv', index=False)

In [14]:
import pandas as pd
from tqdm import tqdm
import time
import os

# Load full movies_content 
movies_content = pd.read_csv('../data/ml-latest-small/processed/movies_content.csv')  

# Load the partial enriched batch you already have
partial_enriched = pd.read_csv('../data/ml-latest-small/processed/movies_fully_enriched_300.csv')  

# Get set of already processed movieIds


# Filter to actually enriched movies (e.g., where 'overview' is not NaN and not empty)
# Adjust column name if your TMDB fields differ
enriched_filter = partial_enriched['overview'].notna() & (partial_enriched['overview'] != '')
done_movie_ids = set(partial_enriched[enriched_filter]['movieId'])

print(f"Actually enriched so far: {len(done_movie_ids)}")  # should be ~300 or less if some failed

# Filter remaining
remaining = movies_content[~movies_content['movieId'].isin(done_movie_ids)].copy()
remaining = remaining[remaining['tmdbId'].notna()]  # only those with tmdbId
remaining['tmdbId'] = remaining['tmdbId'].astype(int)

print(f"Remaining to enrich: {len(remaining)}")
print(f"First remaining tmdbId: {remaining['tmdbId'].iloc[0] if not remaining.empty else 'None'}")

Actually enriched so far: 297
Remaining to enrich: 9437
First remaining tmdbId: 862


In [15]:
batch_size = 400          # adjust down to 200 if timeouts return
sleep_time = 0.45         # ~2.2 req/sec – safe & stable
output_dir = '../data/processed/tmdb_batches'
os.makedirs(output_dir, exist_ok=True)

enriched_all = []  # collect for final merge if you want

start_idx = 0
while start_idx < len(remaining):
    end_idx = min(start_idx + batch_size, len(remaining))
    batch = remaining.iloc[start_idx:end_idx]
    
    print(f"\nStarting batch {start_idx}-{end_idx} ({len(batch)} movies)")
    
    enriched_batch = []
    for idx, row in tqdm(batch.iterrows(), total=len(batch), desc=f"Batch {start_idx//batch_size + 1}"):
        details = fetch_tmdb_details(row['tmdbId'])
        if details:
            details['movieId'] = row['movieId']
            enriched_batch.append(details)
        time.sleep(sleep_time)
    
    if enriched_batch:
        batch_df = pd.DataFrame(enriched_batch)
        batch_file = f"{output_dir}/tmdb_batch_{start_idx}.csv"
        batch_df.to_csv(batch_file, index=False)
        print(f"Saved {len(enriched_batch)} successes to {batch_file}")
        enriched_all.extend(enriched_batch)
    
    start_idx = end_idx
    print(f"Batch done. Pausing 10s before next...")
    time.sleep(10)  # short break between batches

print("\nAll batches complete!")


Starting batch 0-400 (400 movies)


Batch 1:  54%|██████████████████████████████████████▏                                | 215/400 [09:00<07:51,  2.55s/it]

Error fetching 10020: HTTPSConnectionPool(host='api.themoviedb.org', port=443): Read timed out. (read timeout=10)


Batch 1: 100%|███████████████████████████████████████████████████████████████████████| 400/400 [17:14<00:00,  2.59s/it]


Saved 399 successes to ../data/processed/tmdb_batches/tmdb_batch_0.csv
Batch done. Pausing 10s before next...

Starting batch 400-800 (400 movies)


...

Error fetching 502892: 404 Client Error: Not Found for url: https://api.themoviedb.org/3/movie/502892?api_key=be22736f187dfdd6fc2c4ae030458a06&language=en-US&append_to_response=credits%2Ckeywords


Batch 24:  83%|██████████████████████████████████████████████████████████▏           | 197/237 [05:07<00:58,  1.46s/it]

Error fetching 500609: 404 Client Error: Not Found for url: https://api.themoviedb.org/3/movie/500609?api_key=be22736f187dfdd6fc2c4ae030458a06&language=en-US&append_to_response=credits%2Ckeywords


Batch 24: 100%|██████████████████████████████████████████████████████████████████████| 237/237 [06:06<00:00,  1.55s/it]


Saved 222 successes to ../data/processed/tmdb_batches/tmdb_batch_9200.csv
Batch done. Pausing 10s before next...

All batches complete!


In [31]:
import glob

all_batches = [pd.read_csv(f) for f in glob.glob('../data/processed/tmdb_batches/tmdb_batch_*.csv')]
all_new = pd.concat(all_batches, ignore_index=True)

original_partial = pd.read_csv('../data/ml-latest-small/processed/movies_fully_enriched_300.csv')
full_tmdb = pd.concat([original_partial, all_new], ignore_index=True).drop_duplicates(subset=['movieId'])

full_tmdb.to_csv('../data/processed/tmdb_full_enriched.csv', index=False)
print(f"Final enriched movies: {len(full_tmdb)}")

nan_count = full_tmdb['overview'].isna().sum()
print(f"Number of NaN in 'overview': {nan_count}")

Final enriched movies: 9742
Number of NaN in 'overview': 9445


In [32]:
import pandas as pd
import glob
import os

# 1. Paths (adjust if your folder structure is different)
partial_path = '../data/ml-latest-small/processed/movies_fully_enriched_300.csv'
batch_dir    = '../data/processed/tmdb_batches'
batch_pattern = os.path.join(batch_dir, 'tmdb_batch_*.csv')

# 2. Load original partial (the first 300 run)
if os.path.exists(partial_path):
    original_partial = pd.read_csv(partial_path)
    print(f"Loaded original partial: {len(original_partial)} rows")
else:
    print("Original partial file missing!")
    original_partial = pd.DataFrame()

# 3. Load ALL batch files
batch_files = glob.glob(batch_pattern)
print(f"Found {len(batch_files)} batch files")

all_batches = []
for f in batch_files:
    try:
        df = pd.read_csv(f)
        if not df.empty:
            all_batches.append(df)
            print(f"  {os.path.basename(f)}: {len(df)} rows")
        else:
            print(f"  {os.path.basename(f)}: empty file")
    except Exception as e:
        print(f"  Error reading {os.path.basename(f)}: {e}")

if all_batches:
    batches_combined = pd.concat(all_batches, ignore_index=True)
    print(f"Total rows from batches: {len(batches_combined)}")
else:
    batches_combined = pd.DataFrame()
    print("No valid batch data loaded")

# 4. Combine original + all batches, remove duplicates
full_tmdb_data = pd.concat([original_partial, batches_combined], ignore_index=True)
full_tmdb_data = full_tmdb_data.drop_duplicates(subset=['movieId'], keep='last')

print(f"Combined enriched records (unique movieIds): {len(full_tmdb_data)}")
print(f"Non-empty overviews in combined data: {(full_tmdb_data['overview'].notna() & (full_tmdb_data['overview'] != '')).sum()}")

# 5. Merge into full movies list (left join)
movies_content = pd.read_csv('../data/ml-latest-small/processed/movies_content.csv')   # your base file
movies_fully_enriched = movies_content.merge(
    full_tmdb_data, 
    on='movieId', 
    how='left',
    suffixes=('', '_tmdb')
)

# 6. Fill missing enriched columns
enriched_cols = ['overview', 'keywords', 'poster_path', 'directors', 'top_cast', 'release_date', 'vote_average']
for col in enriched_cols:
    if col in movies_fully_enriched.columns:
        movies_fully_enriched[col] = movies_fully_enriched[col].fillna('')

# 7. Rebuild content_text (now with all available TMDB data)
movies_fully_enriched['content_text'] = (
    movies_fully_enriched['genres'].str.replace('|', ' ', regex=False) + ' ' +
    movies_fully_enriched['title'].str.replace(r'\(\d{4}\)', '', regex=True).str.strip() + ' ' +
    movies_fully_enriched['all_tags'].fillna('') + ' ' +
    movies_fully_enriched['overview'].fillna('') + ' ' +
    movies_fully_enriched['keywords'].fillna('') + ' ' +
    movies_fully_enriched['directors'].fillna('') + ' ' +
    movies_fully_enriched['top_cast'].fillna('')
).str.lower().str.replace(r'\s+', ' ', regex=True).str.strip()

# 8. Final checks
print("\nFinal enriched shape:", movies_fully_enriched.shape)
print("NaN in overview (final):", movies_fully_enriched['overview'].isna().sum())
print("Empty or NaN overview:", ((movies_fully_enriched['overview'].isna()) | (movies_fully_enriched['overview'] == '')).sum())

# Check a known enriched movie (e.g. if you know one from a batch)
# Example: print first row with non-empty overview
has_overview = movies_fully_enriched[movies_fully_enriched['overview'].notna() & (movies_fully_enriched['overview'] != '')]
if not has_overview.empty:
    print("\nExample enriched movie:")
    print(has_overview[['movieId', 'title', 'overview']].head(1).to_string(index=False))
else:
    print("No movies with overview found after merge – check batch files content")

Loaded original partial: 9742 rows
Found 24 batch files
  tmdb_batch_0.csv: 399 rows
  tmdb_batch_1200.csv: 399 rows
  tmdb_batch_1600.csv: 244 rows
  tmdb_batch_2000.csv: 400 rows
  tmdb_batch_2400.csv: 399 rows
  tmdb_batch_2800.csv: 398 rows
  tmdb_batch_3200.csv: 398 rows
  tmdb_batch_3600.csv: 400 rows
  tmdb_batch_400.csv: 392 rows
  tmdb_batch_4000.csv: 400 rows
  tmdb_batch_4400.csv: 394 rows
  tmdb_batch_4800.csv: 400 rows
  tmdb_batch_5200.csv: 386 rows
  tmdb_batch_5600.csv: 399 rows
  tmdb_batch_6000.csv: 387 rows
  tmdb_batch_6400.csv: 325 rows
  tmdb_batch_6800.csv: 397 rows
  tmdb_batch_7200.csv: 388 rows
  tmdb_batch_7600.csv: 387 rows
  tmdb_batch_800.csv: 395 rows
  tmdb_batch_8000.csv: 393 rows
  tmdb_batch_8400.csv: 389 rows
  tmdb_batch_8800.csv: 388 rows
  tmdb_batch_9200.csv: 222 rows
Total rows from batches: 9079
Combined enriched records (unique movieIds): 9742
Non-empty overviews in combined data: 9375

Final enriched shape: (9742, 22)
NaN in overview (final):

In [36]:
print(movies_fully_enriched.columns.tolist())

['movieId', 'title', 'genres', 'avg_rating', 'rating_count', 'tmdbId', 'all_tags', 'content_text', 'title_tmdb', 'genres_tmdb', 'avg_rating_tmdb', 'rating_count_tmdb', 'tmdbId_tmdb', 'all_tags_tmdb', 'content_text_tmdb', 'overview', 'keywords', 'poster_path', 'directors', 'top_cast', 'release_date', 'vote_average']


In [37]:
# Overwrite content_text with ALL available features
movies_fully_enriched['content_text'] = (
    # Original features
    movies_fully_enriched['genres'].str.replace('|', ' ', regex=False) + ' ' +
    movies_fully_enriched['title'].str.replace(r'$$   \d{4}   $$', '', regex=True).str.strip() + ' ' +
    movies_fully_enriched['all_tags'].fillna('') + ' ' +
    
    # New TMDB features
    movies_fully_enriched['overview'].fillna('') + ' ' +
    movies_fully_enriched['keywords'].fillna('') + ' ' +
    movies_fully_enriched['directors'].fillna('') + ' ' +
    movies_fully_enriched['top_cast'].fillna('')
).str.lower().str.replace(r'\s+', ' ', regex=True).str.strip()

In [38]:
# Look at a few rows that should have TMDB data
has_tmdb = movies_fully_enriched[movies_fully_enriched['overview'].str.len() > 10]
if not has_tmdb.empty:
    print("Example rows with TMDB data:")
    print(has_tmdb[['movieId', 'title', 'overview', 'content_text']].head(3).to_string(index=False))
else:
    print("No rows with meaningful overview found — something went wrong in merge/combine")

# Check length of content_text for enriched vs non-enriched
print("\nAverage content_text length (characters):")
print(movies_fully_enriched.groupby(movies_fully_enriched['overview'].str.len() > 50)['content_text'].str.len().mean())

Example rows with TMDB data:
 movieId                   title                                                                                                                                                                                                                                                                                                                                                                                                    overview                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               

AttributeError: 'SeriesGroupBy' object has no attribute 'str'

In [39]:
from sklearn.feature_extraction.text import TfidfVectorizer
import scipy.sparse

tfidf = TfidfVectorizer(
    max_features=12000,          # increased because we have more text now
    stop_words='english',
    ngram_range=(1, 2),
    min_df=3,                    # ignore very rare terms
    max_df=0.85                  # ignore super common terms
)

tfidf_matrix = tfidf.fit_transform(movies_fully_enriched['content_text'].fillna(''))

print("New TF-IDF matrix shape:", tfidf_matrix.shape)

# Save
scipy.sparse.save_npz('../data/ml-latest-small/processed/tfidf_matrix_with_tmdb.npz', tfidf_matrix)
movies_fully_enriched.to_csv('../data/ml-latest-small/processed/movies_fully_enriched_with_content_text.csv', index=False)

New TF-IDF matrix shape: (9742, 12000)


In [40]:
# Weight genres by duplicating them (equivalent to ~2x weight)
genre_boost = 2  

movies_fully_enriched['genres_boosted'] = (
    movies_fully_enriched['genres'].str.replace('|', ' ', regex=False) + ' '
) * genre_boost

# Rebuild content_text with boosted genres
movies_fully_enriched['content_text_boosted'] = (
    movies_fully_enriched['genres_boosted'] +
    movies_fully_enriched['title'].str.replace(r'\(\d{4}\)', '', regex=True).str.strip() + ' ' +
    movies_fully_enriched['all_tags'].fillna('') + ' ' +
    movies_fully_enriched['overview'].fillna('') + ' ' +
    movies_fully_enriched['keywords'].fillna('') + ' ' +
    movies_fully_enriched['directors'].fillna('') + ' ' +
    movies_fully_enriched['top_cast'].fillna('')
).str.lower().str.replace(r'\s+', ' ', regex=True).str.strip()

print("Example boosted content_text (first 3):")
print(movies_fully_enriched['content_text_boosted'].head(3))

Example boosted content_text (first 3):
0    adventure animation children comedy fantasy ad...
1    adventure children fantasy adventure children ...
2    comedy romance comedy romance grumpier old men...
Name: content_text_boosted, dtype: str


In [41]:
from sklearn.feature_extraction.text import TfidfVectorizer
import scipy.sparse

tfidf_boosted = TfidfVectorizer(
    max_features=12000,
    stop_words='english',
    ngram_range=(1, 2),
    min_df=3,
    max_df=0.85
)

tfidf_matrix_boosted = tfidf_boosted.fit_transform(
    movies_fully_enriched['content_text_boosted'].fillna('')
)

print("Boosted TF-IDF shape:", tfidf_matrix_boosted.shape)

# Save if you want to compare later
scipy.sparse.save_npz('../data/ml-latest-small/processed/tfidf_matrix_genre_boosted.npz', tfidf_matrix_boosted)

Boosted TF-IDF shape: (9742, 12000)
